In [42]:
import vide.voidUtil as vu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
import pickle

In [43]:
class catalog(object):
    
    def __init__(self, sampledir):
        self.sampleInfo = load_sample(sampledir)
        self.partPos, self.boxLen, self.volNorm, self.isObesrvation, self.ranges, self.extraData = vu.loadPart(sampledir)
        self.voids = load_voids(sampledir)
        
class void(object):
    
    def __init__(self):
        self.macrocenter = None
        self.radius = None
        
        
        
def load_sample(sampledir):
    with open(sampledir+"/sample_info.dat", 'rb') as input:
        sample = pickle.load(input)
    return sample
        
def load_voids(sampledir):
    voids = np.loadtxt(sampledir+"/cleaned_void_big.out")
    voidlist = []
    for i in range(len(voids)):
        _void = void()
        _void.macrocenter = voids[i][:3:]
        _void.radius = voids[i][3]
        voidlist.append(_void)
    return voidlist
        

In [9]:
from vide.backend.classes import *
import numpy as np
import os
import vide.apTools as vp
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d


# -----------------------------------------------------------------------------
def NormProfile(catalog, rMin, rMax, nBins=10, dim=3):

# builds a stacked radial density profile from the given catalog
#   catalog: void catalog
#   rMin: minimum void radius, in Mpc/h
#   rMax: maximum void radius, in Mpc/h
#   nBins: number of bins in profile (detaulf 10)
#
# returns:
#   binCenters: array of radii in binned profile
#   stackedProfile: the stacked density profile
#   sigmas: 1-sigma uncertainty in each bin
  periodicLine = getPeriodic(catalog.sampleInfo)

  print("  Building particle tree...")
  partTree = vu.getPartTree(catalog)

  print("  Selecting voids to stack...")
  voidsToStack = [v for v in catalog.voids if (v.radius > rMin and v.radius < rMax)]

  if len(voidsToStack) == 0:
    print("  No voids to stack!")
    return -1, -1, -1
    
  print("  Stacking voids...")
  allProfiles = []
  for void in voidsToStack:
    center = void.macrocenter
    radius = void.radius
    rMaxProfile = radius*3.0
    
    localPart = catalog.partPos[ vu.getBall(partTree, center, rMaxProfile) ]
    shiftedPart = vu.shiftPart(localPart, center, periodicLine, catalog.ranges)

    if(dim==3):
      dist = np.sqrt(np.sum(shiftedPart[:,:]**2, axis=1))/radius
      thisProfile, norm_radii = np.histogram(dist, bins=nBins, range=(0,3.0))
      radii = norm_radii*radius
      deltaV = 4*np.pi/3*(radii[1:]**3-radii[0:(radii.size-1)]**3)
      thisProfile = np.float32(thisProfile)
      thisProfile /= deltaV
      thisProfile /= catalog.volNorm
      allProfiles.append(thisProfile)
    else:
      dist = np.sqrt(np.sum(shiftedPart[:,0:2]**2, axis=1))
      thisProfile, radii = np.histogram(dist, bins=nBins, range=(0,rMaxProfile))
      deltaV = np.pi*(radii[1:]**2-radii[0:(radii.size-1)]**2)
      thisProfile = np.float32(thisProfile)
      thisProfile /= deltaV
      thisProfile /= catalog.volNorm
      allProfiles.append(thisProfile)
    
    binCenters = 0.5*(norm_radii[1:] + norm_radii[:-1])

  nVoids = len(voidsToStack)
  stackedProfile = np.mean(allProfiles, axis=0) 
  sigmas = np.std(allProfiles, axis=0) / np.sqrt(nVoids)
  
  return binCenters, stackedProfile, sigmas

In [13]:
def Hamas(_r, delta_c, alpha, beta, _rs):
    profile = delta_c*(1-(_r/_rs)**alpha)/(1+(_r)**beta)
    return profile

In [45]:
catalog = catalog("/mnt/data/yuka/output/vide/cola/Om15_2e12ss1.0/sample_0m15_2e12ss1.0_z0.00_d00")

    Loading particle data...


In [46]:
rmin = 25
rmax = 30
binCenters, mean, sigma = NormProfile(catalog, rmin, rmax, nBins=15, dim=3)
lower = mean - sigma
upper = mean + sigma
plt.clf()
vu.fill_between(binCenters, lower-1.0, upper-1.0,
                color='black',
                alpha=0.5,
                )
lineStyle = '-'
    
label = f"radius {rmin} - {rmax}" +"h$^{-1}$Mpc"
plt.plot(binCenters, mean-1.0, lineStyle,
         label=label, color='black',
         linewidth=3)

y = Hamas(binCenters, -0.9, 3.0, 10.0, 1.5)
plt.plot(binCenters, y, '-.',color = 'green')

plt.title(label)
plt.xlabel("$r/r_v$")
plt.ylabel(r"$\rho/\bar{\rho}-1$")

plt.savefig("/mnt/data/yuka/output/vide/cola/figs/big2e12ss1.0/"+f"Om15_Norm_profile_{rmin}-{rmax}"+".png", bbox_inches="tight")
plt.savefig("/mnt/data/yuka/output/vide/cola/figs/big2e12ss1.0/"+f"Om15_Norm_profile_{rmin}-{rmax}"+".pdf", bbox_inches="tight")
plt.show()
plt.clf()

  Building particle tree...
  Selecting voids to stack...
  Stacking voids...


/tmp/ipykernel_2125363/900675692.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [47]:
r_data = binCenters
p0 = [-0.9, 3.0, 10.0, 1.5]
y_obs = mean -1.0
y_err = sigma

params_opt, params_cov = curve_fit(
    Hamas, r_data, y_obs, p0=p0, sigma=y_err, absolute_sigma=True,
    maxfev=10000
)

# 最適フィット値
print("Best fit parameters:")
print(r"$\delta_c$"+f" = {params_opt[0]}")
print(r"$\alpha$" + f"   = {params_opt[1]}")
print(r"$\beta$" + f"    = {params_opt[2]}")
print(r"$r_s/r_v$" + f"     = {params_opt[3]}")

# パラメータの標準偏差（誤差）
param_errors = np.sqrt(np.diag(params_cov))
print("Uncertainties:")
print(r"$\sigma_{\delta_c}$" + f" = {param_errors[0]}")
print(r"$\sigma_{\alpha}$" + f"   = {param_errors[1]}")
print(r"$\sigma_{\beta}$" + f"    = {param_errors[2]}")
print(r"$\sigma_{r_s/r_v}$" + f"      = {param_errors[3]}")

Best fit parameters:
$\delta_c$ = -0.9257306173456836
$\alpha$   = 2.2970786957483122
$\beta$    = 10.67845488977299
$r_s/r_v$     = 1.8724941978458762
Uncertainties:
$\sigma_{\delta_c}$ = 0.028081674419526664
$\sigma_{\alpha}$   = 0.8290910556277244
$\sigma_{\beta}$    = 1.4236434053141938
$\sigma_{r_s/r_v}$      = 0.45045572717847593


In [49]:
plt.plot(binCenters, mean-1.0, lineStyle,
         label="obs", color='black',
         linewidth=3)

vu.fill_between(binCenters, lower-1.0, upper-1.0,
                color='black',
                alpha=0.5,
                )
lineStyle = '-'

#y = Hamas(binCenters, -0.75, 2.0, 8.3, 1.05)
#plt.plot(binCenters, y, '--',color = 'blue', label = "DM",linewidth=1)

y = Hamas(binCenters, params_opt[0], params_opt[1], params_opt[2], params_opt[3])
plt.plot(binCenters, y, '-.',color = 'green', label = "fitting",linewidth=3)

plt.title(label)
plt.xlabel("$r/r_v$")
plt.ylabel(r"$\rho/\bar{\rho}-1$")
plt.legend()

plt.savefig("/mnt/data/yuka/output/vide/cola/figs/big2e12ss1.0/"+f"Om15_Norm_profile_{rmin}-{rmax}"+".png", bbox_inches="tight")
plt.savefig("/mnt/data/yuka/output/vide/cola/figs/big2e12ss1.0/"+f"Om15_Norm_profile_{rmin}-{rmax}"+".pdf", bbox_inches="tight")
plt.show()
plt.clf()

/tmp/ipykernel_2125363/2594746031.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
